In [ ]:
"""
Data Migration Script: Progress DB to MySQL

Purpose:
This script migrates data from a Progress OpenEdge database to a MySQL database.
It fetches data in chunks to optimize memory usage and transfers it efficiently.
The script also ensures secure connection handling and proper table formatting.

Key Features:
- Connects to Progress DB via JDBC using JayDeBeApi
- Reads tables in chunks to prevent memory overflow
- Uses Pandas to process and transfer data to MySQL
- Ensures safe and efficient data transfer with error handling
"""

import pandas as pd
import jaydebeapi
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import os
import time

# Load sensitive data from environment variables
JDBC_DRIVER_JAR = os.getenv("JDBC_DRIVER_JAR_PATH")
JDBC_DRIVER_CLASS = 'com.ddtek.jdbc.openedge.OpenEdgeDriver'
JDBC_URL = os.getenv("PROGRESS_JDBC_URL")
DB_USERNAME = os.getenv("PROGRESS_DB_USER")
DB_PASSWORD = os.getenv("PROGRESS_DB_PASS")

# MySQL connection parameters from environment variables
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT", "3306")
MYSQL_DB = os.getenv("MYSQL_DB")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")

# Create MySQL Engine
MySQL_engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}",
    poolclass=NullPool  # Disable connection pooling
)

# Function to fetch data from Progress DB in chunks
def fetch_data_in_chunks(query, conn, chunk_size=50000):
    cursor = conn.cursor()
    cursor.execute(query)
    columns = [desc[0] for desc in cursor.description]  # Get column names
    while True:
        rows = cursor.fetchmany(chunk_size)
        if not rows:
            break
        yield pd.DataFrame(rows, columns=columns)  # Yield DataFrame chunk

# Function to migrate tables in chunks
def migrate_table_in_chunks(progress_conn, table_name, mysql_engine, chunk_size=50000):
    try:
        query = f'SELECT * FROM "PUB"."{table_name}"'  
        target_table_name = f"{table_name}_PUB"
        
        for chunk_num, chunk in enumerate(fetch_data_in_chunks(query, progress_conn, chunk_size)):
            chunk.to_sql(target_table_name, mysql_engine, index=False, if_exists='append')
            print(f"Chunk {chunk_num + 1} of table '{table_name}' migrated successfully.")
        
        print(f"Table '{table_name}' migration completed successfully.")
    except Exception as e:
        print(f"Error migrating table '{table_name}': {e}")

# Main execution
progress_conn = None
try:
    start_time = time.time()
    
    # Connect to Progress DB
    progress_conn = jaydebeapi.connect(JDBC_DRIVER_CLASS, JDBC_URL, [DB_USERNAME, DB_PASSWORD], JDBC_DRIVER_JAR)
    
    # List of table names to transfer
    table_names_to_transfer = ["DPaySelLineStg", "lgdd_det", "DInvoiceVat", "pi_mstr", "uusg_det", "idh_hist"]

    for table_name in table_names_to_transfer:
        migrate_table_in_chunks(progress_conn, table_name, MySQL_engine)

    end_time = time.time()
    print(f"Data migration completed in {end_time - start_time:.2f} seconds.")
    
finally:
    if progress_conn:
        progress_conn.close()
    MySQL_engine.dispose()
    print("All connections closed.")
